In [1]:
import astropy.units as u, astropy.constants as c
from caltools.caltools import list_calibrators
from astropy.time import Time
import datetime
import dsacalib.constants as ct
from astropy.coordinates import Angle, SkyCoord
import dsacalib.utils as du
import numpy as np
import pandas
from dsacalib.fringestopping import amplitude_sky_model
from dsacalib.fringestopping import pb_resp

2021-03-01 15:59:22,767 Matplotlib created a temporary config/cache directory at /tmp/matplotlib-y9kubsog because the default path (/home/user/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


# new dec: +54:30

Things to add here:
* Flux accounting for beam size
* Flux compared to total flux in the field

In [2]:
pt_dec = 2*u.deg #51.8*u.deg

In [3]:
#pt_dec = 0.415388362*u.rad
today = Time('2021-01-20T00:00:00')
now = Time(datetime.datetime.utcnow())
print(now.sidereal_time('apparent', longitude=ct.OVRO_LON*u.rad).to_value(u.deg))

41.59535665198122


In [4]:
calibrators = []
utc_time = today + np.arange(24*60//5)*5*u.min
for time in utc_time:
    try:
        df = list_calibrators(
            time.sidereal_time('apparent', longitude=ct.OVRO_LON*u.rad).to_value(u.deg),
            pt_dec.to_value(u.deg),
            radius=2.5
        )['NVSS']
        df.sort('flux', reverse=True)
        calibrators += [df[:5]]
    except ConnectionError:
        calibrators += [None]
        print('ConnectionError')

In [5]:
df

NAME,ra,dec,flux,separation
,deg,deg,MJY,
bytes19,float64,float64,float64,float64
NVSS J000622-000425,1.59417,-0.07364,3897.6,128.77435446407696
NVSS J000606+005401,1.52513,0.90031,454.8,72.09906046451636
NVSS J235842+033702,359.67758,3.61747,353.5,126.84500081343518
NVSS J000154+020453,0.47913,2.08142,300.7,34.0172532810653
NVSS J000958+015755,2.49429,1.96533,270.7,87.19618263207187
NVSS J235958+004301,359.99300,0.71703,261.6,99.36729417561925
NVSS J235915+020709,359.81629,2.11919,260.8,73.75646106368723
NVSS J235711+010639,359.29942,1.11089,255.6,117.26589218701048


In [6]:
calibrators_to_keep = []
for i, time in enumerate(utc_time):
    if calibrators[i]['flux'][0]/1e3 > 1:
        calibrators_to_keep += [[
            calibrators[i]['NAME'][0].strip('NVSS '),
            #calibrators[i]['ra'][0]*u.deg,
            #calibrators[i]['dec'][0]*u.deg,
            Angle('{0}h{1}m{2}s'.format(
                calibrators[i]['NAME'][0].strip('NVSS J')[:2],
                calibrators[i]['NAME'][0].strip('NVSS J')[2:4],
                calibrators[i]['NAME'][0].strip('NVSS J')[4:6]
            )).to_value(u.deg)*u.deg,
            Angle('{0}d{1}m{2}s'.format(
                calibrators[i]['NAME'][0].strip('NVSS J')[6:9],
                calibrators[i]['NAME'][0].strip('NVSS J')[9:11],
                calibrators[i]['NAME'][0].strip('NVSS J')[11:13]
            )).to_value(u.deg)*u.deg,
            (np.abs(calibrators[i]['dec'][0]*u.deg-pt_dec)).to(u.deg),
            calibrators[i]['flux'][0]/1e3,
            #calibrators[i]['weighted_flux'][0]/1e3,
            #calibrators[i]['weighted_flux'][0]/sum(calibrators[i]['weighted_flux'])
        ]]
calibrators_df = pandas.DataFrame(
    calibrators_to_keep,
    columns=['source', 'ra', 'dec', 'delta_dec', 'flux (Jy)']#, 'weighted flux (Jy)', 'percent flux']
)
calibrators_df.drop_duplicates(inplace=True)
calibrators_df.reset_index(inplace=True, drop=True)

In [7]:
calibrators_df['weighted flux (Jy)'] = ''
calibrators_df['percent flux'] = ''
for i in range(len(calibrators_df)):
    df = list_calibrators(
        calibrators_df['ra'][i],
        calibrators_df['dec'][i]
    )['NVSS']
    df.sort('flux', reverse=True)
    print(df['NAME'][0], calibrators_df['source'][i])
    if df['NAME'][0].strip('NVSS ') == calibrators_df['source'][i]:
        weighted_flux = []
        for j in range(len(calibrators[i])):
            weighted_flux += [
            df['flux'][j]/1e3*pb_resp(
                    (df['ra'][0]*u.deg).to_value(u.rad),
                    pt_dec.to_value(u.rad),
                    (df['ra'][j]*u.deg).to_value(u.rad),
                    (df['dec'][j]*u.deg).to_value(u.rad),
                    1.4
                )
            ]
        calibrators_df['weighted flux (Jy)'][i] = weighted_flux[0]
        calibrators_df['percent flux'][i] = weighted_flux[0]/sum(weighted_flux)

NVSS J000622-000425 J000622-000425
NVSS J000622-000425 J001310+005142
NVSS J002225+001456 J002225+001456


/home/user/anaconda3/envs/casa/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/user/anaconda3/envs/casa/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/user/anaconda3/envs/casa/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/user/anaconda3/envs/casa/lib/python3.6/site-packa

NVSS J005734-012258 J005905+000651
NVSS J010838+013458 J010838+013458
NVSS J011818+025805 J011818+025805
NVSS J012604-012401 J012528-000556


/home/user/anaconda3/envs/casa/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/user/anaconda3/envs/casa/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/user/anaconda3/envs/casa/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/user/anaconda3/envs/casa/lib/python3.6/site-packa

NVSS J013119+041529 J013119+041529
NVSS J013957+013151 J013957+013151
NVSS J015507+033823 J015507+033823
NVSS J024240-000047 J024240-000047


...

/home/user/anaconda3/envs/casa/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/user/anaconda3/envs/casa/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/user/anaconda3/envs/casa/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/user/anaconda3/envs/casa/lib/python3.6/site-packa

NVSS J173750+032534 J173750+032534
NVSS J175038+031036 J175038+031036
NVSS J182400+014836 J182400+014836
NVSS J185320+011426 J185320+011426
NVSS J190146+011301 J190146+011301
NVSS J192828-010650 J192606+003446


/home/user/anaconda3/envs/casa/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/user/anaconda3/envs/casa/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/user/anaconda3/envs/casa/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/user/anaconda3/envs/casa/lib/python3.6/site-packa

NVSS J193654-002157 J193654-002157
NVSS J195213+023026 J195213+023026
NVSS J213638+004154 J213638+004154
NVSS J214517+043132 J214211+030225
NVSS J221237+015251 J221237+015251
NVSS J231635+040517 J231635+040517
NVSS J234058+043114 J233807+032648


/home/user/anaconda3/envs/casa/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/user/anaconda3/envs/casa/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/user/anaconda3/envs/casa/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/user/anaconda3/envs/casa/lib/python3.6/site-packa

In [8]:
calibrators_df.sort_values(by='flux (Jy)', ascending=False, inplace=True)

In [9]:
#calibrators_df.drop(columns=['delta_dec']).reset_index(drop=True).to_csv('calibrator_sources_+2dec.csv')

In [10]:
mv calibrator_sources_+2dec.csv ..

mv: cannot stat 'calibrator_sources_+2dec.csv': No such file or directory


In [11]:
calibrators_df['dec']

39        2.051388888888889 deg
63       1.2169444444444446 deg
52       1.9633333333333334 deg
12        4.111666666666666 deg
10    -0.013055555555555556 deg
                ...            
47        4.271388888888889 deg
32       0.6277777777777778 deg
54        4.098055555555555 deg
35       3.1527777777777777 deg
27       2.1780555555555554 deg
Name: dec, Length: 72, dtype: object

In [12]:
calibrators_df[calibrators_df['flux (Jy)'] > 3 and abs(calibrators_df['dec']-2]

SyntaxError: invalid syntax (<ipython-input-12-896c851e7160>, line 1)